## Purpose of script:
#### Reviewing Spark dataframe groupby and aggregate operations
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('agg').getOrCreate()

In [3]:
df = spark.read.csv('../Datasets/sales_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [6]:
# useful links for groupby operations:
# https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/
# 
df.groupBy('Company')

In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [8]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [9]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [10]:
df.agg({'Sales': 'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [11]:
# alternative to groupby().max()
group_data = df.groupBy('Company')

group_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [12]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [15]:
df.select(countDistinct('Company')).show()

+-----------------------+
|count(DISTINCT Company)|
+-----------------------+
|                      4|
+-----------------------+



In [16]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [17]:
# renaming the output
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [18]:
from pyspark.sql.functions import format_number

In [20]:
df.select(stddev('Sales').alias('stdd')).show()

+------------------+
|              stdd|
+------------------+
|250.08742410799007|
+------------------+



In [21]:
# formating output to 2 decimal points
sales_stdd = df.select(stddev('Sales').alias('stdd'))

sales_stdd.select(format_number('stdd', 2)).show()

+----------------------+
|format_number(stdd, 2)|
+----------------------+
|                250.09|
+----------------------+



In [22]:
# ordering by column ascending
df.orderBy('Sales').show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
+-------+-------+-----+
only showing top 5 rows



In [23]:
# ordering by column descending
df.orderBy(df['Sales'].desc()).show(5)

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|     FB|  Carl|870.0|
|   APPL|  Mike|750.0|
|   MSFT|  Tina|600.0|
|   APPL| Chris|350.0|
|     FB| Sarah|350.0|
+-------+------+-----+
only showing top 5 rows



In [24]:
spark.stop()